In [1]:
import pandas as pd
import aiohttp
import asyncio
import re
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

C:\Users\Eugene\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
ETHERSCAN_API_KEY = 'GGJU543MXKGFSRTRIGBKEDG8F4CR637M5S' # ввести свой ключ в сети эфириум
BSCSCAN_API_KEY = 'F3ZA4NB4MEGXTWI3FNKZG73RY5CF6ZUU3N' # ввести свой ключ в сети эфириум

In [3]:
async def get_contract_source_code(address, provider) -> dict:
    # create an empty list to hold the source code for each contract
    # source_codes = []
    source_code: dict = {}
    if provider == "trx":
        pass
    else:
        _headers = {
            "Connection": "keep-alive",
            "Accept-Encoding": "gzip, deflate, br",
            "Accept": "*/*",
            "User-Agent": "PostmanRuntime/7.34.0"
        }
        api_key = ETHERSCAN_API_KEY
        base_url = "https://api.etherscan.io/api?module=contract&action=getsourcecode"
        
        if provider == "bsc":
            _headers = {
                "Connection": "keep-alive",
                "Accept-Encoding": "gzip, deflate, br",
                "Accept": "*/*",
            }
            api_key = BSCSCAN_API_KEY
            base_url = "https://api.bscscan.com/api?module=contract&action=getsourcecode"
        
        full_url = f"{base_url}&address={address}&apikey={api_key}"

        async with aiohttp.ClientSession() as session:
            async with session.get(full_url, headers=_headers) as response:
                data = await response.json()
        
        source_code = data["result"][0]
    
    return source_code

# Читаем датасет

In [7]:
df_bsc = pd.read_csv('C:\\Users\\Eugene\\Desktop\\bsc_fishing.csv', sep=';', names=['transact', 'address', 'tag'])
df_bsc['transferFrom'] = ''
df_bsc['transfer'] = ''
df_bsc['_transfer'] = ''
df_bsc

,transact,address,tag,transferFrom,transfer,_transfer
0,0x31d9bb2d2e971f0f2832b32f942828e1f5d82bf9,fishing,scam,,,
1,0x8ed9C7E4D8DfE480584CC7EF45742ac302bA27D7,fishing,scam,,,
2,0x405cFf4cE041d3235E8b1f7AaA4E458998A47363,fishing,scam,,,


In [30]:
df_eth = pd.read_csv('C:\\Users\\Eugene\\Desktop\\eth.csv', sep=';', names=['transact', 'address', 'tag'])
# для других данных
#df_eth = pd.read_csv('C:\\Users\\Eugene\\Desktop\\eth_malicious.csv')
#df_eth = df_eth.drop(columns=['Unnamed: 0'])
df_eth['transferFrom'] = ''
df_eth['transfer'] = ''
df_eth['_transfer'] = ''
df_eth['_transferFrom'] = ''
df_eth_uniq = df_eth.drop_duplicates('transact').reset_index().drop(columns=['index'])
df_eth_uniq

,transact,address,tag,transferFrom,transfer,_transfer,_transferFrom
0,0x6faaeb4050fc4b15b131eff2ea887de829e9fc3f,Fishing,ransom,,,,
1,0x0cc4682b1465bd3bf3b9b13dbd310a1f37c7500d,Fishing,ransom,,,,
2,0x95c8a1f33edaa7e17fd3304fd121f960a21fff1b,Fishing,ransom,,,,
3,0x8d305fd29aebd6e0562de9321d55e7f0e6fbacad,Fishing,ransom,,,,
4,0xa4f5dd69db954c71587173de7d5d7dd330111766,Fishing,ransom,,,,
...,...,...,...,...,...,...,...
296,0x86035a913989d8d2bb631c0bd7657983666168d8,Fishing,ransom,,,,
297,0x903bb9cd3a276d8f18fa6efed49b9bc52ccf06e5,Fishing,ransom,,,,
298,0x702a8dbffac169e5f80e2f5c1cc18b2541887550,Fishing,ransom,,,,
299,0x44525c8dd44b9d470937cc9e7a0275eb14c7b01d,Fishing,ransom,,,,


### Регулярки для вычленения необходимого кода

In [16]:
reForTransferFrom = '(function transferFrom){1}\([^\)]*\)\s+([a-zA-Z]+\s+)*(returns \([^\)]*\)\s+)?\{([^\}]+[^\{]*)+'
_reForTransferFrom = '(function _transferFrom){1}\([^\)]*\)\s+([a-zA-Z]+\s+)*(returns \([^\)]*\)\s+)?\{([^\}]+[^\{]*)+'
reForTransfer = '(function transfer){1}\([^\)]*\)\s+([a-zA-Z]+\s+)*(returns \([^\)]*\)\s+)?\{([^\}]+[^\{]*)+'
_reForTransfer = '(function _transfer){1}\([^\)]*\)\s+([a-zA-Z]+\s+)*(returns \([^\)]*\)\s+)?{([^\}]+[^\{]*)+'

In [17]:
def extractTransfer(allStr):
    str_in_array = allStr.split(sep='}', maxsplit=-1)
    finalTransferFrom = str_in_array[0] + '}'
    for i in range(1, len(str_in_array)):
        if 'function' not in str_in_array[i]:
            finalTransferFrom += str_in_array[i]
            finalTransferFrom += '}'
        else:
            break
    return finalTransferFrom

In [32]:
async def extractFunction(transFunc, transRegex):
    for i in tqdm(range(df_eth_uniq.shape[0])):
        source_code = await get_contract_source_code(df_eth_uniq['transact'][i], 'eth')
        tr = ''
        if source_code['SourceCode'] != '':
            receivedFunc = re.search(transRegex, source_code['SourceCode'])
            if receivedFunc is not None:
                allStr = receivedFunc.group(0)
                tr = extractTransfer(allStr)
            else:
                tr = ''
        df_eth_uniq.loc[i, transFunc] = tr

In [33]:
await extractFunction('transferFrom', reForTransferFrom)
await extractFunction('transfer', reForTransfer)
await extractFunction('_transferFrom', _reForTransferFrom)
await extractFunction('_transfer', _reForTransfer)

100%|██████████| 301/301 [05:22<00:00,  1.07s/it]


NameError: name 'df' is not defined

In [34]:
df_eth_uniq

,transact,address,tag,transferFrom,transfer,_transfer,_transferFrom
0,0x6faaeb4050fc4b15b131eff2ea887de829e9fc3f,Fishing,ransom,,,,
1,0x0cc4682b1465bd3bf3b9b13dbd310a1f37c7500d,Fishing,ransom,,,,
2,0x95c8a1f33edaa7e17fd3304fd121f960a21fff1b,Fishing,ransom,"function transferFrom(address _from, address _...",,,
3,0x8d305fd29aebd6e0562de9321d55e7f0e6fbacad,Fishing,ransom,,,,
4,0xa4f5dd69db954c71587173de7d5d7dd330111766,Fishing,ransom,,,,
...,...,...,...,...,...,...,...
296,0x86035a913989d8d2bb631c0bd7657983666168d8,Fishing,ransom,,,,
297,0x903bb9cd3a276d8f18fa6efed49b9bc52ccf06e5,Fishing,ransom,,,,
298,0x702a8dbffac169e5f80e2f5c1cc18b2541887550,Fishing,ransom,function transferFrom(\r\n address _fro...,,,
299,0x44525c8dd44b9d470937cc9e7a0275eb14c7b01d,Fishing,ransom,,,,


In [33]:
df_eth_uniq.to_csv('eth_malicious_with_transfers.csv', index=False)